In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [ ]:
raw = pd.read_csv('../data/raw/ld_clump_assoc.txt',delimiter='\t')
rename = {'CpG':'cpg',
 'Top SNP':'snp',
 'Beta': 'beta',
 'SE': 'se',
 'Cis/Trans':'cistrans',
 'CpG chr': 'cpg_chr',
 'CpG pos': 'cpg_pos',
 'SNP chr': 'snp_chr',
 'SNP pos': 'snp_pos',
 'P': 'pval'}

raw = data.rename(rename, axis=1)

lst = []
for i in raw['cistrans']:
    if i == 'cis':
        lst.append(True)
    else:
        lst.append(False)
raw['cistrans'] = lst
raw.to_csv('edit_ld_clump_assoc.csv')


In [ ]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12','13']




In [ ]:
# level-based network prototype 
import pandas as pd
from pathlib import Path
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [ ]:
# creating unique nodes for cpg and snp encoding level information 
# level 0 cpg --> query user
# level 1 snp --> cpg-snp
# level 1 cpg --> cpg-snp-cpg connection 

cpg_interest = ['cg13938959','cg23733394']

cpgs = []
snps = []
level = 2

for cpg in cpg_interest:
    
    start = 1
    cpgs.append({'id':cpg,
             'level':0})

    while start <= level:
        
        cpg_included = [cpg['id'] for cpg in cpgs if cpg['level'] < start]
        snps_to_include = data[data['cpg'].isin(cpg_included)]['snp'].values
        [snps.append({'id':snp,'level':start}) for snp in snps_to_include if snp not in [snp['id'] for snp in snps]]
        snps_included = [snp['id'] for snp in snps if snp['level'] == start]
        cpgs_to_include = data[data['snp'].isin(snps_included)]['cpg'].values
        [cpgs.append({'id':cpg,'level':start}) for cpg in cpgs_to_include if cpg not in [cpg['id'] for cpg in cpgs]] 
        start+= 1

edges = data[data['cpg'].isin([cpg['id'] for cpg in cpgs])]
edges = edges[edges['snp'].isin([snp['id'] for snp in snps])]

In [ ]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [ ]:
chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12'] 
 

data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0) 
annot = pd.read_csv('../data/annot/cpg_annotation.csv',index_col=0)
chr_lst = chromosomeList
data['cpg_chr'] = data['cpg_chr'].apply(str)
data['snp_chr'] = data['snp_chr'].apply(str)

data = data[data['cistrans']==False]

data = data.groupby('cpg') # group data by cpgs
data = data.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg

data = data[data['cpg_chr'].isin(chr_lst)]
data = data[data['pval']<=1*10**-minPval]

data['cpg_pos_abs'] = data['cpg_chr'].apply(lambda cpg_chr: chromosome_distance[str(cpg_chr)]) + data['cpg_pos'] # calculate absolute distance of cpg
data['snp_pos_abs'] = data['snp_chr'].apply(lambda snp_chr: chromosome_distance[str(snp_chr)]) + data['snp_pos'] # calculate absolute distance of snp
data['dist'] = abs(data['cpg_pos_abs'] - data['snp_pos_abs']) # calculate distance between pairs
data = data[data['dist']>=minDistance] # filter by min distance

data = data.reset_index(drop=True)

data = data.merge(annot,how='left',on='cpg')
    
data['UCSC_RefGene_Name'] = data['UCSC_RefGene_Name'].fillna('no annotation')

data.drop(['MAPINFO','CHR','cpg_pos_abs','snp_pos_abs','dist'],axis=1,inplace=True)

response = data.to_dict('records')  



In [ ]:
import pandas as pd
ewas_res = pd.read_csv("../data/raw/ewascatalog-results.txt",delimiter="\t")
ewas_stud = pd.read_csv("../data/raw/ewascatalog-studies.txt",delimiter="\t")
ewas_res = ewas_res[['CpG','Chr','Pos','Gene','Beta','SE','P','StudyID']]
ewas_res['Chr'] = ewas_res['Chr']
ewas_res = ewas_res[ewas_res['CpG'].str.startswith('cg')]
ewas_stud = ewas_stud[['Trait','Exposure','N','StudyID','Age','Sex','Ancestry','Tissue']]
ewas_res.to_csv('ewas_results.csv')
ewas_stud.to_csv('ewas_stud.csv')

In [ ]:
# formation of cpg network and integration of ewas results 

data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)
ewas_res = pd.read_csv('ewas_results.csv',dtype={'Chr':str},index_col=0)
ewas_stud = pd.read_csv('ewas_stud.csv',index_col = 0)
cpg_interest = ['cg15580874',
'cg10154826',
'cg01354632',
'cg16677399']
cpgs = []
snps = []
level = 1    
for cpg in cpg_interest:
        
        start = 1
        cpgs.append({'id':cpg,
                'level':0})

        while start <= level:
            
            cpg_included = [cpg['id'] for cpg in cpgs if cpg['level'] < start]
            snps_to_include = data[data['cpg'].isin(cpg_included)]['snp'].values
            [snps.append({'id':snp,'level':start}) for snp in snps_to_include 
                         if snp not in [snp['id'] for snp in snps]]
            snps_included = [snp['id'] for snp in snps if snp['level'] == start]
            cpgs_to_include = data[data['snp'].isin(snps_included)]['cpg'].values
            [cpgs.append({'id':cpg,'level':start}) for cpg in cpgs_to_include 
                         if cpg not in [cpg['id'] for cpg in cpgs]] 
            
            start+= 1

network_df = data[data['cpg'].isin([cpg['id'] for cpg in cpgs])]
network_df = network_df[network_df['snp'].isin([snp['id'] for snp in snps])]
network_df = network_df.reset_index(drop=True)   
network_df['id'] = network_df.index 
edges_data = network_df.to_dict('records')
network_obj = {'nodeAttributes':{'snp':snps,'cpg':cpgs},'edgeAttributes':edges_data}
ewas_res = ewas_res[ewas_res['CpG'].isin(network_df['cpg'].unique())]
ewas_res = ewas_res.merge(ewas_stud,on=["StudyID"],how="left")
ewas_fields = ewas_res.columns.tolist()
ewas_values = ewas_res.values.tolist()
ewas_obj = {"results":ewas_values,"fields":ewas_fields}


In [119]:
minAssoc = 1
minPval = 10
targetChr = 16
startPos = 49879169
endPos = 50479169

data = pd.read_csv('../data/EPICMeQTL.csv',index_col=0)
data = data.query("cistrans == True")
data = data.query(f"snp_chr == {targetChr}")
data = data.query(f"snp_pos > {startPos} & snp_pos < {endPos}")
data = data.groupby('cpg') # group data by cpgs
data = data.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg
data = data.query(f"pval <= {1*10**-minPval}") 


In [ ]:
data

In [127]:
data = pd.read_csv('../data/EPICMeQTL.csv',index_col=0)
annot = pd.read_csv('../data/annot/cpg_annotation.csv',index_col=0)
data['cpg_chr'] = data['cpg_chr'].apply(str)
data['snp_chr'] = data['snp_chr'].apply(str)
data = data.query("cistrans == True")
data = data.query(f"snp_chr == {targetChr}")
data = data.query(f"snp_pos > {startPos} & snp_pos < {endPos}")
data = data.groupby('cpg') # group data by cpgs
data = data.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg
data = data.query(f"pval <= {1*10**-minPval}") 
data = data.reset_index(drop=True)
data = data.merge(annot,how='left',on='cpg')
data.drop(['MAPINFO','CHR'],axis=1,inplace=True)
#data['UCSC_RefGene_Name'] = data['UCSC_RefGene_Name'].fillna('-')
data['id'] = data.index 

response = data.to_dict('records')

               cpg              snp         LD clump cpg_chr   cpg_pos  \
0       cg06325811     1:753405_C_A     1:798400_A_G       1    796328   
1       cg16619049     1:798959_G_A     1:798400_A_G       1    805541   
2       cg13938959    1:838329_G_GC     1:851204_G_C       1    834183   
3       cg13938959     1:930923_A_G     1:935459_A_G       1    834183   
4       cg13938959     1:935222_C_A    1:1022037_C_T       1    834183   
...            ...              ...              ...     ...       ...   
785354  cg09828012  22:51133580_C_A  22:51121416_T_G      22  51164024   
785355  cg09828012  22:51158499_T_G  22:51161019_C_T      22  51164024   
785356  cg19004771  22:51161019_C_T  22:51161019_C_T      22  51173065   
785357  cg20569369  22:51039778_C_A  22:50985703_A_G      22  51173646   
785358  cg09007244  22:51165664_G_A  22:51161019_C_T      22  51221736   

       snp_chr   snp_pos A1  A2    MAF      beta        se          pval  \
0            1    753405  C   A  0.

In [126]:
data

,snp,LD clump,cpg_chr,cpg_pos,snp_chr,snp_pos,A1,A2,MAF,beta,se,pval,FDR,N,n,Effects,cistrans,cpg,UCSC_RefGene_Name,id


In [141]:
minAssoc = 1
minPval = 10
targetChr = 16
startPos = 49879169
endPos = 50479169
data = pd.read_csv('../data/EPICMeQTL.csv',index_col=0)
annot = pd.read_csv('../data/annot/cpg_annotation.csv',index_col=0)
data['cpg_chr'] = data['cpg_chr'].apply(str)
data['snp_chr'] = data['snp_chr'].apply(str)
data = data.query("cistrans == True")
data = data.query(f'snp_chr == "{targetChr}"')
data = data.query(f"snp_pos > {startPos} & snp_pos < {endPos}")

In [142]:
data

,cpg,snp,LD clump,cpg_chr,cpg_pos,snp_chr,snp_pos,A1,A2,MAF,beta,se,pval,FDR,N,n,Effects,cistrans
637325,cg08064228,16:50277438_A_C,16:50277438_A_C,16,49530608,16,50277438,A,C,0.342,0.190784,0.051003,1.850000e-04,4.370000e-02,2,1584,?+??+,True
637550,cg21902192,16:50320962_C_G,16:50312502_C_T,16,49768296,16,50320962,C,G,0.167,0.401584,0.091020,1.040000e-05,4.960000e-03,2,419,???++,True
637583,cg08549806,16:49912619_C_G,16:49912619_C_G,16,49868530,16,49912619,C,G,0.159,-0.358609,0.051312,2.880000e-12,2.280000e-08,2,1531,?-?-?,True
637587,cg15209863,16:49883927_T_A,16:49883927_T_A,16,49885631,16,49883927,T,A,0.215,-0.308998,0.040034,1.230000e-14,0.000000e+00,2,1742,--???,True
637590,cg26747290,16:49886465_C_G,16:49883927_T_A,16,49888100,16,49886465,C,G,0.249,-0.308451,0.035052,1.450000e-18,0.000000e+00,3,1939,---??,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637791,cg16901027,16:50475423_GGCTGAGGGGGC_G,16:50483367_G_A,16,50730221,16,50475423,GGCTGAGGGGGC,G,0.133,-0.228793,0.048377,2.290000e-06,1.470000e-03,2,1742,--???,True
637819,cg08195491,16:50149693_T_C,16:50142679_A_G,16,50746063,16,50149693,T,C,0.104,-0.287314,0.055009,1.800000e-07,1.790000e-04,2,1742,--???,True
637820,cg08195491,16:50348409_A_T,16:50370703_G_C,16,50746063,16,50348409,A,T,0.379,0.162806,0.034713,2.780000e-06,1.720000e-03,2,1742,++???,True
637821,cg08195491,16:50433366_G_A,16:50413232_G_A,16,50746063,16,50433366,G,A,0.168,-0.356120,0.044513,1.300000e-15,0.000000e+00,2,1742,--???,True
